In [1]:
from cmdstanpy import CmdStanModel


import arviz as az
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
_BASE_URL = "https://raw.githubusercontent.com/rmcelreath/rethinking/Experimental/data"
HOWELL_DATASET_PATH = f"{_BASE_URL}/Howell1.csv"
d = pd.read_csv(HOWELL_DATASET_PATH, sep=';', header=0)
d=d[d.age>=18] #just adults 
d.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0


In [4]:
from cmdstanpy import rebuild_cmdstan
rebuild_cmdstan()

Done:  (03:11) | ██████████ | --- CmdStan v2.29.0 built ---         


In [5]:
model_ppc=CmdStanModel(stan_file='height_1_ppc.stan')

R = 1000
sim=model_ppc.sample(iter_sampling=R,
                     iter_warmup=0,
                     chains=1,
                     fixed_param=True,
                     seed=29042020,refresh=R)

INFO:cmdstanpy:compiling stan file /Users/piotrszyller/Documents/2ndLv/s01/DataAnalytics/L04/height_1_ppc.stan to exe file /Users/piotrszyller/Documents/2ndLv/s01/DataAnalytics/L04/height_1_ppc
INFO:cmdstanpy:compiled model executable: /Users/piotrszyller/Documents/2ndLv/s01/DataAnalytics/L04/height_1_ppc
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


## Task1. Prior predictive checks

In [ ]:
model_1_fit=CmdStanModel(stan_file='height_1_fit.stan')

INFO:cmdstanpy:compiling stan file C:\Users\TOSHIBA\Desktop\CPS\Semester 1\Data analysis\Lab4\height_1_fit.stan to exe file C:\Users\TOSHIBA\Desktop\CPS\Semester 1\Data analysis\Lab4\height_1_fit.exe
INFO:cmdstanpy:compiled model executable: C:\Users\TOSHIBA\Desktop\CPS\Semester 1\Data analysis\Lab4\height_1_fit.exe
--- Translating Stan model to C++ code ---
bin/stanc.exe  --o=C:/Users/TOSHIBA/Desktop/CPS/SEMEST~1/DATAAN~1/Lab4/height_1_fit.hpp C:/Users/TOSHIBA/Desktop/CPS/SEMEST~1/DATAAN~1/Lab4/height_1_fit.stan
Warning in 'C:/Users/TOSHIBA/Desktop/CPS/SEMEST~1/DATAAN~1/Lab4/height_1_fit.stan', line 3, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc

--- Compiling, linking C++ code ---
g++ -std=c++1y -m64 -D_REENTRANT -Wall -Wno-unused-function -Wno-uninitialized -Wno-unused-but-

In [ ]:
fit=model_1_fit.sample(data=dict(N=len(d),
                                   heights=d.height.values),
                         seed=28052020)

INFO:cmdstanpy:CmdStan start processing
chain 1 |          | 00:00 Status


chain 1 |▉         | 00:00 Iteration:    1 / 2000 [  0%]  (Warmup)





chain 1 |██████████| 00:00 Sampling completed                       
chain 2 |██████████| 00:00 Sampling completed                       
chain 3 |██████████| 00:00 Sampling completed                       
chain 4 |██████████| 00:00 Sampling completed                       


INFO:cmdstanpy:CmdStan done processing.


In [ ]:
mu = fit.stan_variable('mu')
sigma = fit.stan_variable('sigma')
height = fit.stan_variable('height')

fig, ax = plt.figure()
ax[0][0].hist(mu)
ax[0][1].hist(sigma)
ax[1][1].hist(height)

TypeError: __init__() got an unexpected keyword argument 'subplot'